In [1]:
# Imports
import numpy as np
import pandas as pd
import polars as pl
import pulp
from tqdm.auto import tqdm
import os

from warnings import filterwarnings

filterwarnings("ignore")

# Load master data
vehicles_df = pd.read_csv("../data/Master_data/vehicle_master_data.csv")
distances_df = pd.read_csv("../data/Master_data/distance_master_data_2.csv")
# Read input
input_df = pd.read_csv("../data/Input_data/Input5.csv")

In [2]:
def clean_data_mode(df):
    # Drop mode distance that are not compatible to the specific vehicle
    df.drop(
        df[(df["Air Distance (km)"].isna()) & (df["Transportation modes"] == "Air")].index,
        axis=0,
        inplace=True,
    )
    df.drop(
        df[(df["Rail Distance (km)"].isna()) & (df["Transportation modes"] == "Rail")].index,
        axis=0,
        inplace=True,
    )
    df.drop(
        df[(df["Road Distance (km)"].isna()) & (df["Transportation modes"] == "Road")].index,
        axis=0,
        inplace=True,
    )
    df.reset_index(inplace=True, drop=True)
    
    return df

def pre_calculate_data_features(df):
    # Pre-calculate time, cost and obj function value
    time_list = []
    cost_list = []
    obj1_list = []

    for i in range(len(df)):
        if df["Transportation modes"][i] == "Air":
            time_list.append(df["Air Distance (km)"][i] / df["Avg speed (km/hr)"][i])
            cost_list.append(df["Air Distance (km)"][i] * df["Cost per km"][i])
            obj1_list.append(df["Air Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)

        elif df["Transportation modes"][i] == "Road":
            time_list.append(df["Road Distance (km)"][i] / df["Avg speed (km/hr)"][i])
            cost_list.append(df["Road Distance (km)"][i] * df["Cost per km"][i])
            obj1_list.append(df["Road Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)

        else:
            time_list.append(df["Rail Distance (km)"][i] / df["Avg speed (km/hr)"][i])
            cost_list.append(df["Rail Distance (km)"][i] * df["Cost per km"][i])
            obj1_list.append(df["Rail Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)


    df["Time (hrs)"] = time_list
    df["Cost"] = cost_list
    df["obj1"] = obj1_list

def prepare_model_data(vehicles_df, distances_df):

    # Createing decision variable column in the dataframe
    df = distances_df.join(vehicles_df, how="cross")
    df["decisions"] = df["Source"] + ", " + df["Destination"] + ", " + df["Transportation modes"] + ", " + df["Vehicle_type"]

    df = clean_data_mode(df)

    pre_calculate_data_features(df)

    # Filtering the model data according to provided input data
    i1 = df.set_index(list(df[["Source", "Destination"]])).index
    i2 = input_df.set_index(list(input_df[["From", "To"]])).index

    df0 = df[i1.isin(i2)]
    df0.reset_index(drop=True, inplace=True)

    # pandas -> polars (for faster performance)
    df1 = pl.from_pandas(df0)
    return df, df1

In [3]:
def master_data_preparation(vehicles_df, demands_df):
    # master availability mapping
    master_availability = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Availability"]))

    # master demand mapping
    master_demand = {}
    for i, j in zip(
        np.array(demands_df[["Source", "Destination"]].drop_duplicates()),
        [
            0,
        ]
        * len(demands_df[["Source", "Destination"]].drop_duplicates()),
    ):
        master_demand[tuple(i)] = j
    
    return master_availability, master_demand


def prepare_demand_data(input_df, master_demand):
    # real/input demand mapping
    demand = {}
    for i, j in zip(np.array(input_df[["From", "To"]]), input_df["Quantity (MT)"]):
        demand[tuple(i)] = j

    for k in master_demand.keys():
        if k not in demand.keys():
            demand[k] = 0
    
    return demand

In [4]:
def create_optim_model(df1, input_df):
    # Result lists choices
    vehicle_count_choice_list = []
    material_transport_quantity_choice_list = []
    co2_emission_choice_list = []
    transportation_cost_choice_list = []
    transportation_time_choice_list = []

    # Result lists trips/return trips
    vehicle_count_trip_list = []
    material_transport_quantity_trip_list = []
    co2_emission_trip_list = []
    transportation_cost_trip_list = []
    transportation_time_trip_list = []

    ##########################################################
    # ------------------- Model Definition ------------------#
    ##########################################################
    # Create a LP minimization problem
    model = pulp.LpProblem("Co2e Optimization", pulp.LpMinimize)

    ############################################################
    # ------------------- Decision Variables ------------------#
    ############################################################

    # choice decision variable -> Bool
    choice_decision = pulp.LpVariable.dicts(
        "Choice_Decision_",
        (tuple(df1["decisions"][i].split(", ")) + (j,) for i in range(len(df1)) for j in range(master_availability[df1["decisions"][i].split(", ")[-1]])),
        lowBound=0,
        cat="Binary",
    )

    # trip/return trip decision variable -> Int
    trip_decision = pulp.LpVariable.dicts(
        "Trip_Decision_",
        (tuple(df1["decisions"][i].split(", ")) + (j,) for i in range(len(df1)) for j in range(master_availability[df1["decisions"][i].split(", ")[-1]])),
        lowBound=0,
        cat="Integer",
    )

    ############################################################
    # ------------------- Objective Function ------------------#
    ############################################################

    obj_list = []
    for s, d, m, v, c in choice_decision.keys():
        obj_list.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["obj1"].item())
        obj_list.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["obj1"].item())

    model += pulp.lpSum(obj_list)

    #####################################################
    # ------------------- Constraints ------------------#
    #####################################################

    # ========== Demand Constraint ==========
    (s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
    col = []

    for s, d, m, v, c in choice_decision.keys():
        if (s0, d0) == (s, d):
            col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
            col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
        else:
            if demand[(s0, d0)] == 0:
                model += pulp.lpSum(col) == demand[(s0, d0)]
                (s0, d0) = (s, d)
                col = []
                col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
                col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
            elif demand[(s0, d0)] >= 0:
                model += pulp.lpSum(col) >= demand[(s0, d0)]
                (s0, d0) = (s, d)
                col = []
                col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
                col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())

    if demand[(s0, d0)] == 0:
        model += pulp.lpSum(col) == demand[(s0, d0)]
    elif demand[(s0, d0)] >= 0:
        model += pulp.lpSum(col) >= demand[(s0, d0)]


    # ========== Minimum Quantity Allowed Constraint ==========

    (s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
    col = []

    for s, d, m, v, c in choice_decision.keys():
        if (s0, d0) == (s, d):
            col.append(
                choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item()
            )
            col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item())
        else:
            if demand[(s0, d0)] == 0:
                model += pulp.lpSum(col) == demand[(s0, d0)]
                (s0, d0) = (s, d)
                col = []
                col.append(
                    choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item()
                )
                col.append(
                    trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item()
                )
            elif demand[(s0, d0)] >= 0:
                model += pulp.lpSum(col) <= demand[(s0, d0)]
                (s0, d0) = (s, d)
                col = []
                col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
                col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())

    if demand[(s0, d0)] == 0:
        model += pulp.lpSum(col) == demand[(s0, d0)]
    elif demand[(s0, d0)] >= 0:
        model += pulp.lpSum(col) <= demand[(s0, d0)]


    # ========== Time Constraint ==========

    for s, d, m, v, c in choice_decision.keys():
        if input_df[(input_df["From"] == s) & (input_df["To"] == d)].shape[0]:
            exp = (choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Time (hrs)"].item()) + (
                2.5 * trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Time (hrs)"].item()
            )
            model += pulp.LpConstraint(
                e=pulp.LpAffineExpression(exp),
                sense=pulp.LpConstraintLE,
                rhs=input_df[(input_df["From"] == s) & (input_df["To"] == d)]["Time (hrs)"].item(),
            )


    # ========== Cost Constraint ==========

    (s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
    col = []

    for s, d, m, v, c in choice_decision.keys():
        if ((s0, d0) == (s, d)) & input_df[(input_df["From"] == s) & (input_df["To"] == d)].shape[0]:
            col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
            col.append( trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
        elif (s0, d0) != (s, d):
            if input_df[(input_df["From"] == s0) & (input_df["To"] == d0)].shape[0]:
                model += pulp.lpSum(col) <= input_df[(input_df["From"] == s0) & (input_df["To"] == d0)]["Cost"]
                (s0, d0) = (s, d)
                col = []
                col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
                col.append( trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
            else:
                (s0, d0) = (s, d)
                col = []

    if input_df[(input_df["From"] == s) & (input_df["To"] == d)].shape[0]:
        model += pulp.lpSum(col) <= input_df[(input_df["From"] == s0) & (input_df["To"] == d0)]["Cost"]

    # cost_1=col
    # ========== Other Constraints ==========

    for s, d, m, v, c in choice_decision.keys():
        model += trip_decision[(s, d, m, v, c)] <= 10000000 * choice_decision[(s, d, m, v, c)]

    for s, d, m, v, c in choice_decision.keys():
        if m != "Road":
            model += trip_decision[(s, d, m, v, c)] == 0
            
    return model, choice_decision, trip_decision

In [5]:
def extract_output(model):
    # Extracting necessary output info. from the solver
    mt_choice = []
    mt_trip = []
    for v in model.variables():
        if v.varValue != 0:
            if v.name.split("__(")[0] == "Choice_Decision":
                mt_choice.append(
                    v.name.split("__(")[1].rstrip(")").split(",_")
                    + [
                        v.varValue,
                    ]
                )
            else:
                mt_trip.append(
                    v.name.split("__(")[1].rstrip(")").split(",_")
                    + [
                        v.varValue,
                    ]
                )

    mt_choice_output = pd.DataFrame(
        mt_choice,
        columns=[
            "Source",
            "Destination",
            "Transportation mode",
            "Vehicle type",
            "Vehicle number",
            "Count",
        ],
    )
    mt_choice_output_df = mt_choice_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
    mt_choice_output_df["S_D_M"] = (
        mt_choice_output_df["Source"] + "_" + mt_choice_output_df["Destination"] + "_" + mt_choice_output_df["Transportation mode"]
    )

    mt_trip_output = pd.DataFrame(
        mt_trip,
        columns=[
            "Source",
            "Destination",
            "Transportation mode",
            "Vehicle type",
            "Vehicle number",
            "Count",
        ],
    )
    mt_trip_output_df = mt_trip_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
    mt_trip_output_df["S_D_M"] = mt_trip_output_df["Source"] + "_" + mt_trip_output_df["Destination"] + "_" + mt_trip_output_df["Transportation mode"]
    mt_trip_output_df["Vehicle type with number"] = mt_trip_output_df["Vehicle type"] + "_" + mt_trip_output_df["Vehicle number"]
    
    
    return mt_choice_output_df, mt_trip_output_df


def create_mappings(df, distances_df, vehicles_df):
    # Apply all mapping to the output
    dist_list = []
    vals = []
    for c in distances_df.columns[2:]:
        for i in range(len(distances_df)):
            dist_list.append(distances_df["Source"][i] + "_" + distances_df["Destination"][i] + "_" + c.split(" ")[0])
            vals.append(distances_df[c][i])

    distance_mappings = dict(zip(dist_list, vals))
    quantity_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Capacity (metric tons)"]))
    emission_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Co2e (g/km)"]))
    cost_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Cost per km"]))
    time_mappings = dict(
        zip(
            (df["Source"] + "_" + df["Destination"] + "_" + df["Transportation modes"] + "_" + df["Vehicle_type"].str.replace(" ", "_")),
            df["Time (hrs)"],
        )
    )
    
    return {"distance_mappings":distance_mappings, "quantity_mappings":quantity_mappings, "emission_mappings":emission_mappings, "cost_mappings":cost_mappings, "time_mappings":time_mappings}

def apply_mappings(mappings, df, t=None):
    # Get output data ready before putting into presentable format
    df["Loaded quantity"] = df["Vehicle type"].str.replace("_", " ").map(mappings["quantity_mappings"])
    df["Distance"] = df["S_D_M"].map(mappings["distance_mappings"])
    df["Emission"] = (
        (df["Vehicle type"].str.replace("_", " ").map(mappings["emission_mappings"])) * df["Distance"] / 1000
    )
    df["Transportation cost"] = (df["Vehicle type"].str.replace("_", " ").map(mappings["cost_mappings"])) * df[
        "Distance"
    ]
    df["Time (hrs)"] = (df["S_D_M"] + "_" + df["Vehicle type"]).map(mappings["time_mappings"])
    if t==("trip" or "return" or "return trip"):
        df["Time (hrs)"] *= df["Count"]
    

In [6]:
def generate_output(df):
    # Pivot tables for more presentable data

    vehicle_count = pd.pivot_table(
        df,
        values="Count",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    vehicle_count["Total Vehicles in Use"] = vehicle_count.iloc[:, 2:].sum(axis=1)

    material_transport_quantity = pd.pivot_table(
        df,
        values="Loaded quantity",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    material_transport_quantity["Total Quantity Transported"] = material_transport_quantity.iloc[:, 2:].sum(axis=1)

    co2_emission = pd.pivot_table(
        df,
        values="Emission",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    co2_emission["Total co2 Emission (kg)"] = co2_emission.iloc[:, 2:].sum(axis=1)

    transportation_cost = pd.pivot_table(
        df,
        values="Transportation cost",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    transportation_cost["Total transportation cost"] = transportation_cost.iloc[:, 2:].sum(axis=1)

    transportation_time = pd.pivot_table(
        df,
        values="Time (hrs)",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="max",
    ).reset_index()
    transportation_time["Total Time (hrs)"] = transportation_time.iloc[:, 2:].sum(axis=1)

    # Some cleaning

    vehicle_count.fillna(0, inplace=True)
    material_transport_quantity.fillna(0, inplace=True)
    co2_emission.fillna(0, inplace=True)
    transportation_cost.fillna(0, inplace=True)
    transportation_time.fillna(0, inplace=True)

    return vehicle_count, material_transport_quantity, co2_emission, transportation_cost, transportation_time


In [7]:
def save_outputs(folder_path):
    os.makedirs(folder_path, exist_ok=True)
    with pd.ExcelWriter(f"{folder_path}/Vehicle_Count.xlsx") as writer1:
        vehicle_count_choice.to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        vehicle_count_trip.to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=vehicle_count_choice.shape[1] + 2)

    with pd.ExcelWriter(f"{folder_path}/Material_Transport_Quantity.xlsx") as writer2:
        material_transport_quantity_choice.to_excel(writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        material_transport_quantity_trip.to_excel(
            writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=material_transport_quantity_choice.shape[1] + 2
        )

    with pd.ExcelWriter(f"{folder_path}/CO2_Emission.xlsx") as writer3:
        co2_emission_choice.to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        co2_emission_trip.to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=co2_emission_choice.shape[1] + 2)

    with pd.ExcelWriter(f"{folder_path}/Transportation_Cost.xlsx") as writer4:
        transportation_cost_choice.to_excel(writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        transportation_cost_trip.to_excel(
            writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_cost_choice.shape[1] + 2
        )

    with pd.ExcelWriter(f"{folder_path}/Transportation_Time.xlsx") as writer5:
        transportation_time_choice.to_excel(writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        transportation_time_trip.to_excel(
            writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_time_choice.shape[1] + 2
        )


In [8]:
def prepare_and_save_summary(folder_path):
    os.makedirs(folder_path, exist_ok=True)
    output_large_merged = mt_choice_output_df.merge(mt_trip_output_df[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count', 'Time (hrs)']], on=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number'], how="left").fillna(-1)
    output_large_merged['Count_y']+=1
    output_large_merged['Count_y'] = output_large_merged[['Count_x', 'Count_y']].max(axis=1)
    output_large_merged['Time (hrs)'] = output_large_merged[['Time (hrs)_x', 'Time (hrs)_y']].max(axis=1)
    summary_large = output_large_merged[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count_y', 'Loaded quantity', 'Emission', 'Transportation cost', 'Time (hrs)']]
    summary_large.rename({'Count_y': 'Number of trips', 'Emission': 'Total CO2 Emission', 'Transportation cost': 'Total Transportation Cost', 'Time (hrs)': 'Total Transportation Time (hrs)'}, axis=1, inplace=True)
    summary_large['Vehicle number'] = summary_large['Vehicle number'].astype('int64')

    summary_large['Loaded quantity'] *= summary_large['Number of trips']
    summary_large['Total CO2 Emission'] *= (2*summary_large['Number of trips'])
    summary_large['Total Transportation Cost'] *= (2*summary_large['Number of trips'])
    summary_large['Total Transportation Time (hrs)'] *= (2*summary_large['Number of trips']-1)

    result_df = summary_large.groupby(['Source', 'Destination']).agg({
         'Vehicle type': 'count',
        'Total CO2 Emission': 'sum',
        'Total Transportation Time (hrs)': 'max',
        'Total Transportation Cost': 'sum',
        'Loaded quantity':'sum',

    }).reset_index()
    output_short_merged = result_df.merge(input_df, left_on=['Source', 'Destination'], right_on=['From', 'To'])
    summary_short = output_short_merged[['Source', 'Destination', 'Quantity (MT)', 'Time (hrs)', 'Cost', 'Loaded quantity', 'Total Transportation Time (hrs)', 'Total Transportation Cost', 'Total CO2 Emission']]
    summary_short.rename({'Quantity(MT)': 'Demand', 'Time (hrs)': 'Time Constraint (hrs)', 'Cost': 'Cost Constraint', 'Total transportation time': 'Total transportation time (hrs)', 'Loaded quantity': 'Total Quantity Transported'}, axis=1, inplace=True)    

    with pd.ExcelWriter(f"{folder_path}/Summary.xlsx") as writer:
        summary_large.to_excel(writer, sheet_name="Summary", index=False, startrow=0, startcol=0)
        summary_short.to_excel(writer, sheet_name="Summary", index=False, startrow=summary_large.shape[0]+3, startcol=0)
        
    return summary_large, summary_short

In [9]:
%%time
df0, df1 = prepare_model_data(vehicles_df, distances_df)
master_availability, master_demand = master_data_preparation(vehicles_df, df0)
demand = prepare_demand_data(input_df, master_demand)
model, choice_decision, trip_decision = create_optim_model(df1, input_df)

CPU times: total: 10.5 s
Wall time: 3.33 s


In [10]:
col=[]
for s, d, m, v, c in choice_decision.keys():
    col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
    col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())

In [11]:
model.solve()
print(pulp.LpStatus[model.status])
mt_choice_output_df, mt_trip_output_df = extract_output(model)
mappings = create_mappings(df0, distances_df, vehicles_df)
apply_mappings(mappings, mt_choice_output_df)
apply_mappings(mappings, mt_trip_output_df, t='return')
vehicle_count_choice, material_transport_quantity_choice, co2_emission_choice, transportation_cost_choice, transportation_time_choice = generate_output(mt_choice_output_df)
vehicle_count_trip, material_transport_quantity_trip, co2_emission_trip, transportation_cost_trip, transportation_time_trip = generate_output(mt_trip_output_df)
save_outputs(f"../data/Output_data/v4/ans1")
summary_large, summary_short = prepare_and_save_summary(f"../data/Output_data/v4/ans1/result")

Optimal


In [12]:
####################################################################
#------------------- Solve for second iteration -------------------#
####################################################################
first_best_co2_emission = pulp.value(model.objective)
model += (model.objective >= first_best_co2_emission+0.00001)
total_cost = summary_short['Total Transportation Cost'].sum() - 1   
model += pulp.lpSum(col) <= total_cost

In [14]:
model.solve()
print(pulp.LpStatus[model.status])
mt_choice_output_df, mt_trip_output_df = extract_output(model)
mappings = create_mappings(df0, distances_df, vehicles_df)
apply_mappings(mappings, mt_choice_output_df)
apply_mappings(mappings, mt_trip_output_df, t='return')
vehicle_count_choice, material_transport_quantity_choice, co2_emission_choice, transportation_cost_choice, transportation_time_choice = generate_output(mt_choice_output_df)
vehicle_count_trip, material_transport_quantity_trip, co2_emission_trip, transportation_cost_trip, transportation_time_trip = generate_output(mt_trip_output_df)
save_outputs(f"../data/Output_data/v4/ans2")
summary_large, summary_short = prepare_and_save_summary(f"../data/Output_data/v4/ans2/result")

Optimal


In [15]:
###################################################################
#------------------- Solve for third iteration -------------------#
###################################################################
second_best_co2_emission = pulp.value(model.objective)
model += (model.objective >= second_best_co2_emission+0.00001)
total_cost = summary_short['Total Transportation Cost'].sum() - 1
model += pulp.lpSum(col) <= total_cost

In [16]:
# model.solve()
# print(pulp.LpStatus[model.status])
# mt_choice_output_df, mt_trip_output_df = extract_output(model)
# mappings = create_mappings(df0, distances_df, vehicles_df)
# apply_mappings(mappings, mt_choice_output_df)
# apply_mappings(mappings, mt_trip_output_df, t='return')
# vehicle_count_choice, material_transport_quantity_choice, co2_emission_choice, transportation_cost_choice, transportation_time_choice = generate_output(mt_choice_output_df)
# vehicle_count_trip, material_transport_quantity_trip, co2_emission_trip, transportation_cost_trip, transportation_time_trip = generate_output(mt_trip_output_df)
# save_outputs(f"../data/Output_data/v4/ans3")
# summary_large, summary_short = prepare_and_save_summary(f"../data/Output_data/v4/ans3/result")

1035.98400995718*Choice_Decision__('Delhi',_'Chennai',_'Air',_'CARGO_PLANE',_0) + 1035.98400995718*Choice_Decision__('Delhi',_'Chennai',_'Air',_'CARGO_PLANE',_1) + 177.82230352623944*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_10',_0) + 177.82230352623944*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_10',_1) + 177.82230352623944*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_10',_2) + 171.72084605269265*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_12',_0) + 171.72084605269265*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_12',_1) + 185.80535383053032*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_5',_0) + 185.80535383053032*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_5',_1) + 191.63974597114984*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_6A',_0) + 191.63974597114984*Choice_Decision__('Delhi',_'Chennai',_'Rail',_'GOODS_RAIL_WAG_6A',_1) + 130.57390154255012*